In [11]:
import time
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import division
from scipy.misc import toimage
import scipy.io as spio

from lasso_codes.lasso import lasso_fista
from lasso_codes.lasso import lasso_fista_col
from lasso_codes.lasso import col_to_image
from lasso_codes.lasso import compute_lasso_grad
from lasso_codes.lasso import compute_lasso_grad_col2


In [2]:
from cs231n.data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=490, num_validation=10, num_test=100):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (490, 32, 32, 3)
Train labels shape:  (490,)
Validation data shape:  (10, 32, 32, 3)
Validation labels shape:  (10,)
Test data shape:  (100, 32, 32, 3)
Test labels shape:  (100,)


In [12]:
#Gradient testing Col

tf.reset_default_graph()
X0= tf.placeholder(tf.float64, [75,None])
D1=tf.placeholder(tf.float64, [75,50])

A1= lasso_fista_col(Dict=D1,
                 X=X0,
                 lambda1=0.01,
                 maxItr=500,
                 NonNeg=True,
                 debug=False)


X0_tmp= tf.placeholder(tf.float64, [75, None])
D1_tmp=tf.placeholder(tf.float64, [75, 50])
A1_tmp=tf.placeholder(tf.float64, [50, None])
grad_output1=tf.placeholder(tf.float64, [50, None])

[lambda1_grad, X_grad, D_grad]= compute_lasso_grad_col2(X=X0_tmp,
                                                       A=A1_tmp,
                                                       D=D1_tmp,
                                                       lambda1=0.01,
                                                       grad_output=grad_output1,               
                                                       NonNeg=True)



eta=10**(-5)
error9=[]
with tf.Session() as sess:
    with tf.device("/gpu:0"):
        tf.global_variables_initializer().run()
        for itr in range(100):
            start_time = time.time()
            images=np.random.rand(75,10000).astype('float64')
            noise=np.zeros([75,10000]).astype('float64')
            i=np.random.randint(0,75)
            j=np.random.randint(0,10000)
            noise[i,j]=eta
            D1_np=np.random.rand(75, 50).astype('float64')
            
            feed_dict = {X0: images+noise, D1: D1_np}
            variables=A1
            A1_np_eta_plus = sess.run(variables,feed_dict=feed_dict)
            
            
            
            feed_dict = {X0: images-noise, D1: D1_np}
            variables=A1
            A1_np_eta_min = sess.run(variables,feed_dict=feed_dict)
            
            
            feed_dict = {X0: images, D1: D1_np}
            variables=A1
            start_time = time.time()
            A1_np = sess.run(variables,feed_dict=feed_dict)
            print("--- %s seconds ---Lasso" % (time.time() - start_time))
            
            grad_output1_np=np.random.rand(50,10000).astype('float64')
            grad_emp=np.sum(np.multiply((A1_np_eta_plus - A1_np_eta_min)/(2*eta), grad_output1_np ) )
            
            
            feed_dict = {X0_tmp: images, D1_tmp: D1_np,
                         A1_tmp: A1_np, grad_output1: grad_output1_np}
            variables=[lambda1_grad, X_grad, D_grad]
            start_time = time.time()
            [lambda1_grad_np, X_grad_np, D_grad_np] = sess.run(variables,feed_dict=feed_dict)
            print("--- %s seconds ---Grad" % (time.time() - start_time))
            grad_closed = X_grad_np[i, j]
            
            error9.append( np.absolute(1 - grad_emp/grad_closed)*100 )
            print(i, j)
            print("emirical grad=" + str(grad_emp) +",  computed grad=" + str(grad_closed) +",  error="+  str(error9[itr]))
           


--- 0.09448719024658203 seconds ---Lasso
--- 0.1074981689453125 seconds ---Grad
47 3298
emirical grad=-0.004807104070465065,  computed grad=-0.00448663185017942,  error=7.142824082453503
--- 0.09133291244506836 seconds ---Lasso
--- 0.037648916244506836 seconds ---Grad
30 7342
emirical grad=0.029985313249631863,  computed grad=0.02979911267157966,  error=0.6248527602292997
--- 0.09419083595275879 seconds ---Lasso
--- 0.051375389099121094 seconds ---Grad
66 3348
emirical grad=-0.020667609180359536,  computed grad=-0.025604504394698524,  error=19.281354320457712
--- 0.09280729293823242 seconds ---Lasso
--- 0.04837846755981445 seconds ---Grad
9 2422
emirical grad=-0.017495348185638245,  computed grad=-0.01725870584139773,  error=1.3711476770922992
--- 0.0942695140838623 seconds ---Lasso
--- 0.05732154846191406 seconds ---Grad
5 9254
emirical grad=-0.020488441345252864,  computed grad=-0.020520467709115205,  error=0.15607034067802728
--- 0.09312200546264648 seconds ---Lasso
--- 0.0583889484

--- 0.09293913841247559 seconds ---Lasso
--- 0.046123504638671875 seconds ---Grad
16 6021
emirical grad=0.022889108823248867,  computed grad=0.022717123039706934,  error=0.7570755471162371
--- 0.0926048755645752 seconds ---Lasso
--- 0.041129350662231445 seconds ---Grad
68 1279
emirical grad=0.03246828995082052,  computed grad=0.03220051717646185,  error=0.831579110643621
--- 0.09366488456726074 seconds ---Lasso
--- 0.051856040954589844 seconds ---Grad
3 8144
emirical grad=0.01499340518619037,  computed grad=0.015362461702351657,  error=2.4023266798757437
--- 0.09422945976257324 seconds ---Lasso
--- 0.04607105255126953 seconds ---Grad
61 2852
emirical grad=0.07402568813154566,  computed grad=0.07407190972347022,  error=0.062400972375520336
--- 0.09380578994750977 seconds ---Lasso
--- 0.04254603385925293 seconds ---Grad
51 6650
emirical grad=0.05621190884841393,  computed grad=0.05624744646041459,  error=0.0631808450640925
--- 0.09334278106689453 seconds ---Lasso
--- 0.04326891899108887 

--- 0.09227895736694336 seconds ---Lasso
--- 0.05404210090637207 seconds ---Grad
24 4582
emirical grad=-0.002290569806767756,  computed grad=-0.002466634104133814,  error=7.1378360118751765
--- 0.09397149085998535 seconds ---Lasso
--- 0.04558086395263672 seconds ---Grad
26 4137
emirical grad=0.036994401500081285,  computed grad=0.03707625347913912,  error=0.22076658609503363
--- 0.0909278392791748 seconds ---Lasso
--- 0.04556536674499512 seconds ---Grad
48 9101
emirical grad=-0.09281705824827156,  computed grad=-0.09313099652313678,  error=0.3370932198574983
--- 0.0921635627746582 seconds ---Lasso
--- 0.0531008243560791 seconds ---Grad
7 5954
emirical grad=0.07117462027749978,  computed grad=0.07142945852738448,  error=0.3567691189861222
--- 0.09296941757202148 seconds ---Lasso
--- 0.05133223533630371 seconds ---Grad
23 9899
emirical grad=0.08658700234309769,  computed grad=0.0867203308913277,  error=0.15374543300243904
--- 0.09421992301940918 seconds ---Lasso
--- 0.04630088806152344 s

In [13]:
#Gradient testing

#images=X_train[0:10,:,:,:]
#from lasso_codes.lasso import  lasso_fista
tf.reset_default_graph()
X0= tf.placeholder(tf.float64, [None,32,32,3])
D1=tf.placeholder(tf.float64, [27,50])
A1= lasso_fista(Dict=D1,
                 input=X0,
                 lambda1=0.01,
                 ksizes=[1,3,3,1],
                 strides=[1,1,1,1],
                 rates=[1,1,1,1],
                 padding='SAME',
                 maxItr=500,
                 NonNeg=True,
                 debug=False,
                 name='lasso')

X0_tmp= tf.placeholder(tf.float64, [None,32,32,3])
D1_tmp=tf.placeholder(tf.float64, [27,50])
A1_tmp=tf.placeholder(tf.float64, [None,32,32,50])
grad_output1=tf.placeholder(tf.float64, [None,32,32,50])

[lambda1_grad, X_grad, D_grad]= compute_lasso_grad(X=X0_tmp,
                 A=A1_tmp,
                 D=D1_tmp,
                 lambda1=0.01,
                 grad_output=grad_output1,               
                 ksizes=[1,3,3,1],
                 strides=[1,1,1,1],
                 rates=[1,1,1,1],
                 padding='SAME',
                 params=[10,32,32,3],
                 NonNeg=True)


eta=10**(-5)
error=[]
with tf.Session() as sess:
    with tf.device("/gpu:0"):
        tf.global_variables_initializer().run()
        for itr in range(100):
            
            images=np.random.rand(10,32,32,3).astype('float64')
            noise=np.zeros([10,32,32,3]).astype('float64')
            i=np.random.randint(0,10)
            j=np.random.randint(0,32)
            c=np.random.randint(0,32)  #c
            b=np.random.randint(0,3)
            noise[i,j,c,b]=eta
            D1_np=np.random.rand(27, 50).astype('float64')
            
            feed_dict = {X0: images+noise, D1: D1_np}
            variables=A1
            A1_np_eta_plus = sess.run(variables,feed_dict=feed_dict)
            
            
            feed_dict = {X0: images-noise, D1: D1_np}
            variables=A1
            A1_np_eta_min = sess.run(variables,feed_dict=feed_dict)
            
            
            feed_dict = {X0: images, D1: D1_np}
            variables=A1
            start_time = time.time()
            A1_np = sess.run(variables,feed_dict=feed_dict)
            print("--- %s seconds ---Lasso" % (time.time() - start_time))
            
            grad_output1_np=np.random.rand(10,32,32, 50).astype('float64')
            grad_emp=np.sum(np.multiply((A1_np_eta_plus - A1_np_eta_min)/(2*eta), grad_output1_np ) )
            
            
            feed_dict = {X0_tmp: images, D1_tmp: D1_np,
                         A1_tmp: A1_np, grad_output1: grad_output1_np}
            variables=[lambda1_grad, X_grad, D_grad]
            start_time = time.time()
            [lambda1_grad_np, X_grad_np, D_grad_np] = sess.run(variables,feed_dict=feed_dict)
            print("--- %s seconds ---Grad" % (time.time() - start_time))
            grad_closed = X_grad_np[i, j, c, b]
            
            error.append( np.absolute(1 - grad_emp/grad_closed)*100 )
            print(i, j, c, b)
            print("emirical grad=" + str(grad_emp) +",  computed grad=" + str(grad_closed) +",  error="+  str(error[itr]))
            
        
print(A1_np_eta_plus.shape)
print(D1_np.shape)
print(images.shape)

#from matplotlib import pyplot
#bins = np.linspace(0, 1000, 20)
#pyplot.hist(error, bins, alpha=0.5, label='error')
#pyplot.hist(error9, bins, alpha=0.5, label='error9')

#pyplot.legend(loc='upper right')
#pyplot.show()

--- 0.09533095359802246 seconds ---Lasso
--- 1.069457769393921 seconds ---Grad
1 14 14 0
emirical grad=0.46397678572067386,  computed grad=0.41451956002035906,  error=11.931216393717525
--- 0.09440970420837402 seconds ---Lasso
--- 0.06403040885925293 seconds ---Grad
9 16 6 2
emirical grad=1.2330396686065277,  computed grad=1.28957184362811,  error=4.383794148493003
--- 0.09453272819519043 seconds ---Lasso
--- 0.06737470626831055 seconds ---Grad
6 4 21 2
emirical grad=0.1468103878387707,  computed grad=0.1555363591084454,  error=5.610245295500738
--- 0.09436869621276855 seconds ---Lasso
--- 0.06827211380004883 seconds ---Grad
6 12 30 2
emirical grad=0.8441468986255583,  computed grad=0.8725992431336332,  error=3.260642813062542
--- 0.09276175498962402 seconds ---Lasso
--- 0.06577730178833008 seconds ---Grad
8 10 13 0
emirical grad=0.6596880942391775,  computed grad=0.6663145686184123,  error=0.9944963972459253
--- 0.09407973289489746 seconds ---Lasso
--- 0.07292747497558594 seconds ---G

--- 0.09226655960083008 seconds ---Lasso
--- 0.0563967227935791 seconds ---Grad
9 18 21 2
emirical grad=-0.7010399275441912,  computed grad=-0.7261668828968069,  error=3.4602177466947825
--- 0.0957791805267334 seconds ---Lasso
--- 0.07125329971313477 seconds ---Grad
6 24 17 2
emirical grad=0.3606246527577038,  computed grad=0.34355364545186146,  error=4.968949545969603
--- 0.09451723098754883 seconds ---Lasso
--- 0.0627138614654541 seconds ---Grad
8 3 14 0
emirical grad=0.15405987004356414,  computed grad=0.17029644873556393,  error=9.534302572105846
--- 0.09522104263305664 seconds ---Lasso
--- 0.0755162239074707 seconds ---Grad
2 27 0 0
emirical grad=0.6407148280158836,  computed grad=0.6298088747526127,  error=1.7316290227816777
--- 0.09233927726745605 seconds ---Lasso
--- 0.07012677192687988 seconds ---Grad
1 17 3 2
emirical grad=0.22983552197555046,  computed grad=0.24942581864897084,  error=7.85415751245494
--- 0.09283447265625 seconds ---Lasso
--- 0.06923961639404297 seconds ---G

--- 0.09526538848876953 seconds ---Lasso
--- 0.07237911224365234 seconds ---Grad
0 19 29 1
emirical grad=0.21272383187781313,  computed grad=0.2272673447169248,  error=6.399297205336074
--- 0.09206962585449219 seconds ---Lasso
--- 0.07242655754089355 seconds ---Grad
3 10 14 2
emirical grad=0.3538078152725942,  computed grad=0.3598118755933578,  error=1.668666524934026
--- 0.09512162208557129 seconds ---Lasso
--- 0.06103944778442383 seconds ---Grad
9 17 11 1
emirical grad=0.00901636257386687,  computed grad=0.021286716914550224,  error=57.64324479880752
--- 0.09472775459289551 seconds ---Lasso
--- 0.0965580940246582 seconds ---Grad
4 12 22 1
emirical grad=-0.11970461632574916,  computed grad=-0.13976043562173673,  error=14.350140801126921
--- 0.09356045722961426 seconds ---Lasso
--- 0.07283473014831543 seconds ---Grad
0 26 20 2
emirical grad=-0.19500552712343266,  computed grad=0.3067935674380341,  error=163.5624562639566
--- 0.09428095817565918 seconds ---Lasso
--- 0.08034276962280273 

In [ ]:
#Test Lasso Fista reconstruction
#mat = spio.loadmat('test.mat', squeeze_me=True)

X_np=np.random.rand(10,32,32,3)
D_np=np.random.rand(27,50)

print(X_np.shape)
print(D_np.shape)


tf.reset_default_graph()
X0= tf.placeholder(tf.float64, [None,32,32,3])
D1=tf.placeholder(tf.float64, [27,50])
A1= lasso_fista(Dict=D1,
                 input=X0,
                 lambda1=0,
                 ksizes=[1,3,3,1],
                 strides=[1,3,3,1],
                 rates=[1,1,1,1],
                 padding='SAME',
                 maxItr=500,
                 NonNeg=False,
                 debug=False,
                 name='lasso')

#A1=tf.transpose(A1,[0,2,1,3])
A1_col=tf.reshape(A1,[-1,50])
A1_col=tf.transpose(A1_col)
Xp_rec=tf.matmul(D1,A1_col)

X_rec= col_to_image(Xp_rec,ksizes=[1,3,3,1],strides=[1,3,3,1],rates=[1,1,1,1],padding='SAME', target_h=32,
                         target_w=32, target_dep=3, bsize=10)
nrm = col_to_image(tf.ones_like(Xp_rec),ksizes=[1,3,3,1],strides=[1,3,3,1],rates=[1,1,1,1],padding='SAME', target_h=32,
                       target_w=32, target_dep=3, bsize=10)
X_rec=tf.divide(X_rec,nrm)



feed_dict = {X0: X_np, D1: D_np}
variables=[X_rec,A1]
with tf.Session() as sess:
    with tf.device("/gpu:0"):
        tf.global_variables_initializer().run()
        [X_rec_np, A1_np] = sess.run(variables,feed_dict=feed_dict)

print(A1_np.shape)
print(X_rec_np.shape)
#print(X_rec_np - X_np)
print( np.linalg.norm(np.reshape(X_rec_np - X_np,[1,-1]),2)  )
print( np.linalg.norm(np.reshape( X_np,[1,-1]),2)  )
print( np.linalg.norm(np.reshape(X_rec_np,[1,-1]),2)  )
#Xp_rec_np=np.matmul(D_np, A1_np, out=None)

In [ ]:
print(A1_np[0,0,0,:])

In [ ]:
#Test Lasso Fista
mat = spio.loadmat('test.mat', squeeze_me=True)

X_mat = mat['Y'] # array
D_mat = mat['A'] # structure containing an array
A_mat = mat['X_clean'] # array of structures
lambda1=mat['lambda']

X_mat=np.transpose(X_mat,(3,0,1,2))
D_mat=np.reshape(D_mat,[27,50])
A_mat=np.transpose(A_mat,(3,0,1,2))

print(X_mat.shape)
print(D_mat.shape)
print(A_mat.shape)


tf.reset_default_graph()
X0= tf.placeholder(tf.float64, [None,32,32,3])
D1=tf.placeholder(tf.float64, [27,50])
A1= lasso_fista(Dict=D1,
                 input=X0,
                 lambda1=0.01,
                 ksizes=[1,3,3,1],
                 strides=[1,1,1,1],
                 rates=[1,1,1,1],
                 padding='SAME',
                 maxItr=50,
                 NonNeg=True,
                 debug=False,
                 name='lasso')

feed_dict = {X0: X_mat, D1: D_mat}
variables=A1
with tf.Session() as sess:
    with tf.device("/gpu:0"):
        tf.global_variables_initializer().run()
        A1_np = sess.run(variables,feed_dict=feed_dict)

print('------------------')
print(A1_np.shape)
print(A_mat.shape)

E= A1_np -A_mat
print(np.linalg.norm (np.reshape (E,[-1,1]),2))
#print(np.linalg.norm (np.reshape (A1_np,[-1,1]),2))
#print(np.linalg.norm (np.reshape (A_mat,[-1,1]),2))

#print(A1_np[0,0,1,:])
#print(A_mat[0,1,0,:])
#print(np.linalg.norm (np.reshape (A1_np[0,:,:,1] - A_mat[0,:,:,1],[-1,1]),2) )
#print(np.linalg.norm (np.reshape (A1_np[0,:,:,1],[-1,1]),2))
#print(np.linalg.norm (np.reshape (A_mat[0,:,:,1],[-1,1]),2))



In [ ]:
#Test Lasso Fista_col
mat = spio.loadmat('test_lasso_col.mat', squeeze_me=True)
X_mat = mat['X'] # array
D_mat = mat['D'] # structure containing an array
A_mat = mat['A'] # array of structures
print(A_mat.shape)
print(D_mat.shape)
print(X_mat.shape)

tf.reset_default_graph()
X0= tf.placeholder(tf.float64, [27,None])
D1=tf.placeholder(tf.float64, [27,50])

A1= lasso_fista_col(Dict=D1,
                 X=X0,
                 lambda1=0.01,
                 maxItr=50,
                 NonNeg=True,
                 debug=False)

feed_dict = {X0: X_mat, D1: D_mat}
variables=A1

with tf.Session() as sess:
    with tf.device("/gpu:0"):
        tf.global_variables_initializer().run()
        A1_np = sess.run(variables,feed_dict=feed_dict)
#print(np.linalg.norm (np.reshape (A1_np[0,:,:,1] - A_mat[0,:,:,1],[-1,1]),2) )

print(A1_np.shape)
#print(A1_np.shape)

print(np.linalg.norm(np.reshape(A1_np-A_mat,[-1,1]),2))

E_np=X_mat-np.matmul(D_mat,A1_np)
print(np.linalg.norm(np.reshape(E_np,[-1,1]),2))

E_mat=X_mat-np.matmul(D_mat,A_mat)
print(np.linalg.norm(np.reshape(E_mat,[-1,1]),2))

In [ ]:
# Test Recon

In [ ]:
# Test Recon
#from lasso_codes.lasso import  lasso_reconstruct
tf.reset_default_graph()

A1= tf.placeholder(tf.float32, [None,32,32,50])
D1=tf.placeholder(tf.float32, [27,50])
res= lasso_reconstruct(Dict=D1_np, A=A1_np,
                       ksizes=[1,3,3,1],
                       strides=[1,1,1,1],
                       rates=[1,1,1,1],
                       padding='SAME')


variables=res

with tf.Session() as sess:
    with tf.device("/gpu:0"):
        tf.global_variables_initializer().run()
        X_rec = sess.run(variables)
print(X_rec.shape)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=False, figsize=(12,4))

ax1.imshow(toimage(X_rec[0,:,:,:]))
ax2.imshow(toimage(images[0,:,:,:]))            
            
f.tight_layout()
plt.show()

In [ ]:
   @ops.RegisterGradient("LassoFistaGrad")
   def _lasso_fista_identity_grad(op, grad):
      #print('helloooo')
      #print(op.inputs[0])


      print(len(op.inputs))
      D = op.inputs[0]
      X = op.inputs[1]
      lambda1=op.inputs[2]
      A = op.inputs[3]
      ksizes=[1,3,3,1]
      strides=[1,1,1,1]
      rates=[1,1,1,1]
      pad=1
      dims=[10,32,32,3]

      if (pad==0):
         padding='VALID'
      else:
         padding='SAME'
      #print('hello')
      #print(tf.shape(grad))
      #print('Bye')

      [lambda_grad, X_grad, D_grad]= compute_lasso_grad(X=X, A=A, D=D, lambda1=lambda1, grad_output=grad,
                                                            ksizes=ksizes, strides=strides, rates=rates, padding=padding, dims=dims)

      #D_grad=D
      #X_grad=X
      lambda_grad=tf.expand_dims(lambda_grad,0)
      #print(lambda_grad)
      #lambda_grad=lambda1
      #print(lambda_grad)
      A_grad=grad

      return[D_grad, X_grad, lambda_grad, A_grad,None,None,None,None, None]
    
    
    
     g = tf.get_default_graph()
   #with g.as_default():
   with g.gradient_override_map({'LassoFista': 'LassoFistaGrad'}):
      A=_lasso_fista(Dict,input,lambda1,ksizes,strides,rates,padding,maxItr,NonNeg, debug,name)
      dims=[10,32,32,3]
      if (padding=='VALID'):
         pad=0
      else:
         pad=1
      _module=tf.load_op_library(tf.sysconfig.get_lib()+'/python/user_ops/lasso_fista.so')
      A_n=_module.lasso_fista(Dict, input, lambda1, A, ksizes, strides, rates, pad, dims)

      #[A_n,Dict_n,lambda1_n]=tf.tuple([A,Dict,lambda1],name="NoOp")
      #A_idn=_lasso_fista_identity(Dict,input,lambda1,A,ksizes,strides,rates,padding,maxItr=50,NonNeg=True, debug=False, name='lassoIdn')

      A_out=A_n + tf.stop_gradient(A-A_n)
      #grad = tf.gradients(A_out, Dict)
      #print(A_out.name)
   return(A_out)

In [ ]:
print(pad_per_col_np.shape)

In [ ]:
print(pad_per_col_np[0:100,:])

In [ ]:
print(A1_np[:,0])